# Analyzing CIA Factbook Data Using SQL

This project will focus on working with the data from the [CIA World Factbook site](https://www.cia.gov/the-world-factbook/), which contains a compendium of statistics about all of the coountries on Earth.

Information contained in the data set is:

- `population` - the global population
- `population_growth` - the annual population growth rate, as a percentage
- `area` - the total land and water area

The purpose of the project will be to explore the data set using SQL. The database will be analyzed using SQLite.

# Loading in Database and Entry Analysis

In [1]:
#connecting jupyter to Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [3]:
#showing tables in database

In [4]:
%%sql
select *
from sqlite_master
where type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [5]:
#Listing first 5 rows of facts table to see the general information and columns

In [6]:
%%sql
select *
from facts
limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


After loading in our `facts` table, we can see the different columns and their values:

- `name`-Name of the country
- `area`-Total land and sea area of the country
- `population`-Country's population
- `population_growth`-Country's population growth as a percentage
- `birth_rate`-Country's birth rate, or the number of births a year per 1,000 people
- `death_rate`-Country's death rate, number of death a year per 1,000 people
- `area_land`-Country's land area in square kilometers
- `area_water`-Country's waterarea in square kilometers

# Summary Statistics

We start to explore more into the data of the facts table.

In [7]:
#query returning the minimum population, maximum population, minimum population growth, and maximum population growth

In [8]:
%%sql
select min(population) as min_pop,
       max(population) as max_pop, 
       min(population_growth) as min_pop_growth,
       max(population_growth) as max_pop_growth
from facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


From the results we can see that:

- there is a country with a minimum population of 0
- there is a population of 7,256,490,011 where the total population on earth is roughly the same amount

Using subqueries we can focus on these countries without using the specific values.

# Exploring the Outliers

We dive into the outliers and see if the information is correct.

In [9]:
#Finding the country with a population 0

In [10]:
%%sql
select *
from facts
where population==(select min(population) from facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


The country with the population of 0 is Antarctica. After a bit a research, the country has no indigenous habitat which would cause the following results.

In [11]:
#Finding the country with a population of over 7.2 billion

In [12]:
%%sql
select *
from facts
where population==(select max(population) from facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We can see from the results that `World` is entered at the country giving the characteristics of the total world population.

Now that we know that `World` is an entry, we should start to review the database while excluding the row for `World`.

In [13]:
#finding min population, max population, min population growth, max population growth excluding 'world' row

In [14]:
%%sql
select min(population) as min_pop,
       max(population) as max_pop, 
       min(population_growth) as min_pop_growth,
       max(population_growth) as max_pop_growth
from facts
where name<>'World';

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


After filtering out the `World` row, the largest population is 1,367,485,388

In [15]:
#selecting row with max population excluding 'world'

In [16]:
%%sql
select *
from facts
where population == (select max(population) from facts where name<>'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


We find out that the country with the largest population is China.

In [17]:
#selecting row with highest groth rate

In [26]:
%%sql
select *
from facts
where population_growth == (select max(population_growth) from facts where name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan has the highest population growth rate at 4.02.

# Average Population and Area

In [19]:
#selecting row with highest groth rate

In [21]:
#finding avg population and avg area excluding 'world' row

In [22]:
%%sql
select avg(population) as avg_population, avg(area) as avg_area
from facts
where name<>'World';

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


The results show that the average population is a little over 32 million and the avg area is 555 thousand square kilometers.

# Finding Densely Populated Countries

We build on the query above to find countries that a

In [23]:
#finding countries that population above the average and area below the average

In [24]:
%%sql
select *
from facts
where population > (select avg(population) from facts where name <> 'World')
and area < (select avg(area) from facts where name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


We can see that 14 different countries fit the criteria we searched for, where their population is above the average and their area ia below the average.

# Countries with highest population/area ratio

In [ ]:
#finding and limiting countries with highest population/area ratio by 10 in descending order

In [37]:
%%sql
select *, population/area as ratio
from facts
where name <> 'World'
order by ratio DESC
limit 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,ratio
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299


Countries that have the highest population to area ratio. Notice that most of these are not countries but islands or independent states.

# Water to Land Ratio

In [45]:
%%sql 
select *, CAST(area_water as float)/CAST(area_land as float) as water_to_land_ratio
from facts
where name <> 'World'
order by water_to_land_ratio DESC
limit 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_to_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6666666666666
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67,4.520231213872832
246,rq,Puerto Rico,13791,8870,4921,3598357,0.6,10.86,8.67,8.15,0.5547914317925592
12,bf,"Bahamas, The",13880,10010,3870,324597,0.85,15.5,7.05,0.0,0.3866133866133866
71,pu,Guinea-Bissau,36125,28120,8005,1726170,1.91,33.38,14.33,0.0,0.2846728307254623
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,0.25939625850340137
125,nl,Netherlands,41543,33893,7650,16947904,0.41,10.83,8.66,1.95,0.22571032366565366
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,0.22292237442922375
56,er,Eritrea,117600,101000,16600,6527689,2.25,30.0,7.52,0.0,0.16435643564356436
99,li,Liberia,111369,96320,15049,4195666,2.47,34.41,9.69,0.0,0.15623961794019933


Top 10 countries with highest water to land ratio.

In [41]:
%%sql
select *
from facts
where (area_water > area_land) and (name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


Countries that have more water than land

# Countries with the most addition to population

In [ ]:
#sorting countries that will add the most poeple to their populations next year

In [51]:
%%sql
select *, CAST(population_growth as float) + CAST(birth_rate as float) as population_addition
from facts
where name <> 'World'
order by population_addition DESC
limit 10;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,population_addition
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56,48.7
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26,47.97
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74,47.03
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0,45.29
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0,45.06
194,za,Zambia,752618,743398,9220,15066266,2.88,42.13,12.67,0.68,45.010000000000005
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0,44.88
160,so,Somalia,637657,627337,10320,10616380,1.83,40.45,13.62,8.49,42.28
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46,41.56
121,mz,Mozambique,799380,786380,13000,25303113,2.45,38.58,12.1,1.98,41.03


# Countries that have a higher death rate than birth rate

In [52]:
%%sql
select *
from facts
where death_rate > birth_rate and name <> 'World'

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


# Conclusion

We conducted analysis on the data from the CIA Factbook site. We answered questions such as which country had the largest and smallest population, the average area and population, etc, as well as outliers within the data set. Many insights were found and basic usage of SQL was shown.